# 03 作者画像与影响力分析

**【优化新增】分析维度**:
1. **作者立场分布**: 按影响力分层（followers）分析立场构成
2. **立场一致性**: 对比作者bio vs 实际推文立场
3. **影响力分析**: Top 50高影响力作者的叙事与情感偏好

In [1]:
import sys
from pathlib import Path

project_root = Path('/workspace')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f"✅ Python 路径已配置: {project_root}")

✅ Python 路径已配置: /workspace


## 步骤 1: 加载并合并数据

In [2]:
import polars as pl
from src import io

# 加载完整数据（包含作者立场预标注）
enriched_df = pl.read_parquet("../parquet/tweets_enriched.parquet")
print(f"📊 完整数据: {enriched_df.height:,} 条推文, {enriched_df.width} 列")

# 【诊断】检查关键字段
print(f"\n🔍 检查关键字段:")
print(f"  author_stance_prelabel 存在: {'author_stance_prelabel' in enriched_df.columns}")
print(f"  author_stance_confidence 存在: {'author_stance_confidence' in enriched_df.columns}")

if 'author_stance_prelabel' not in enriched_df.columns:
    print(f"\n❌ 错误: enriched parquet 缺少 author_stance_prelabel 字段")
    print(f"   请确认已重新运行 00_data_intake.ipynb 的所有步骤")
    print(f"   当前字段列表 (前30个):")
    for i, col in enumerate(enriched_df.columns[:30], 1):
        print(f"     {i:2d}. {col}")
    raise ValueError("Missing author_stance_prelabel field in enriched data")

# 加载内容分析结果（包含情感、叙事、立场分类）
content_df = pl.read_parquet("../parquet/content_analysis.parquet")
print(f"\n📊 内容分析数据（采样）: {content_df.height:,} 条推文")

# 合并：将内容分析的字段 join 到完整数据上（基于 pseudo_id）
merged_df = enriched_df.join(
    content_df.select([
        'pseudo_id', 'political_stance', 'stance_confidence',
        'primary_emotion', 'primary_narrative'
    ]),
    on='pseudo_id',
    how='left'
)

print(f"\n✅ 数据合并完成: {merged_df.height:,} 条推文")
print(f"   author_stance_prelabel 存在: {'author_stance_prelabel' in merged_df.columns}")
print(f"   political_stance 存在: {'political_stance' in merged_df.columns}")
print(f"   内容分析覆盖率: {merged_df.filter(pl.col('political_stance').is_not_null()).height / merged_df.height * 100:.1f}%")

📊 完整数据: 508,954 条推文, 28 列

🔍 检查关键字段:
  author_stance_prelabel 存在: True
  author_stance_confidence 存在: True

📊 内容分析数据（采样）: 4,000 条推文

✅ 数据合并完成: 508,954 条推文
   author_stance_prelabel 存在: True
   political_stance 存在: True
   内容分析覆盖率: 0.8%


## 步骤 2: 影响力分层分析

In [3]:
# 只统计有内容分析结果的推文
analyzed_df = merged_df.filter(pl.col('political_stance').is_not_null())
print(f"📊 用于作者分析的推文数: {analyzed_df.height:,}")

# 按作者聚合
author_stats = analyzed_df.group_by('pseudo_author_userName').agg([
    pl.len().alias('tweet_count'),
    pl.col('author_followers').first().alias('followers'),
    pl.col('author_stance_prelabel').first().alias('bio_stance'),
    pl.col('author_stance_confidence').first().alias('bio_confidence'),
    pl.col('political_stance').mode().first().alias('tweet_stance_mode'),
    pl.col('stance_confidence').mean().alias('avg_stance_confidence'),
    pl.col('retweetCount').sum().alias('total_retweets'),
    pl.col('likeCount').sum().alias('total_likes')
]).filter(pl.col('followers').is_not_null())

# 影响力分层
author_stats = author_stats.with_columns(
    pl.when(pl.col('followers') >= 1_000_000)
      .then(pl.lit('Mega (1M+)'))
      .when(pl.col('followers') >= 100_000)
      .then(pl.lit('High (100K-1M)'))
      .when(pl.col('followers') >= 10_000)
      .then(pl.lit('Medium (10K-100K)'))
      .otherwise(pl.lit('Low (<10K)'))
      .alias('influence_tier')
)

print(f"\n✅ 作者画像完成: {author_stats.height:,} 位作者")
print(f"\n影响力分层:")
print(author_stats.group_by('influence_tier').agg(pl.len().alias('count')).sort('count', descending=True))

📊 用于作者分析的推文数: 4,000

✅ 作者画像完成: 135 位作者

影响力分层:
shape: (3, 2)
┌───────────────────┬───────┐
│ influence_tier    ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ High (100K-1M)    ┆ 88    │
│ Medium (10K-100K) ┆ 28    │
│ Mega (1M+)        ┆ 19    │
└───────────────────┴───────┘


## 步骤 3: 立场一致性分析

In [4]:
# 计算bio立场 vs 推文立场的一致性
author_stats = author_stats.with_columns(
    pl.when(
        (pl.col('bio_stance') == pl.col('tweet_stance_mode')) & 
        (pl.col('bio_stance') != 'neutral')
    ).then(pl.lit('一致'))
    .when(pl.col('bio_stance') == 'neutral')
    .then(pl.lit('bio无立场'))
    .otherwise(pl.lit('不一致'))
    .alias('stance_consistency')
)

print("📊 立场一致性统计:")
consistency_dist = author_stats.group_by('stance_consistency').agg(
    pl.len().alias('count')
).sort('count', descending=True)
print(consistency_dist)

# 不一致案例
inconsistent = author_stats.filter(
    pl.col('stance_consistency') == '不一致'
).sort('followers', descending=True)

print(f"\n🔍 立场不一致作者: {inconsistent.height} 位")
if inconsistent.height > 0:
    print("\nTop 5 高影响力不一致案例:")
    print(inconsistent.select([
        'pseudo_author_userName', 'followers', 'bio_stance', 
        'tweet_stance_mode', 'tweet_count'
    ]).head(5))

📊 立场一致性统计:
shape: (3, 2)
┌────────────────────┬───────┐
│ stance_consistency ┆ count │
│ ---                ┆ ---   │
│ str                ┆ u32   │
╞════════════════════╪═══════╡
│ bio无立场          ┆ 114   │
│ 不一致             ┆ 15    │
│ 一致               ┆ 6     │
└────────────────────┴───────┘

🔍 立场不一致作者: 15 位

Top 5 高影响力不一致案例:
shape: (5, 5)
┌────────────────────────┬───────────┬──────────────┬───────────────────┬─────────────┐
│ pseudo_author_userName ┆ followers ┆ bio_stance   ┆ tweet_stance_mode ┆ tweet_count │
│ ---                    ┆ ---       ┆ ---          ┆ ---               ┆ ---         │
│ i64                    ┆ i64       ┆ str          ┆ str               ┆ u32         │
╞════════════════════════╪═══════════╪══════════════╪═══════════════════╪═════════════╡
│ 980608787796177        ┆ 3495134   ┆ conservative ┆ neutral           ┆ 1           │
│ 976617995024402        ┆ 1258812   ┆ conservative ┆ neutral           ┆ 1           │
│ 690114391314736        ┆ 551138    ┆ 

## 步骤 4: Top 50 高影响力作者分析

In [5]:
# 提取 Top 50
top_50 = author_stats.sort('followers', descending=True).head(50)

print(f"📊 Top 50 影响力作者:")
print(f"  总followers: {top_50['followers'].sum():,}")
print(f"  总推文数: {top_50['tweet_count'].sum():,}")
print(f"  平均followers: {top_50['followers'].mean():.0f}")

# 立场分布
print(f"\n立场分布:")
stance_dist = top_50.group_by('tweet_stance_mode').agg(
    pl.len().alias('count')
).sort('count', descending=True)
print(stance_dist)

# 显示 Top 10
print(f"\nTop 10:")
print(top_50.select([
    'pseudo_author_userName', 'followers', 'tweet_count',
    'bio_stance', 'tweet_stance_mode', 'influence_tier'
]).head(10))

📊 Top 50 影响力作者:
  总followers: 137,827,264
  总推文数: 251
  平均followers: 2756545

立场分布:
shape: (2, 2)
┌───────────────────┬───────┐
│ tweet_stance_mode ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ neutral           ┆ 49    │
│ conservative      ┆ 1     │
└───────────────────┴───────┘

Top 10:
shape: (10, 6)
┌────────────────────┬───────────┬─────────────┬──────────────┬───────────────────┬────────────────┐
│ pseudo_author_user ┆ followers ┆ tweet_count ┆ bio_stance   ┆ tweet_stance_mode ┆ influence_tier │
│ Name               ┆ ---       ┆ ---         ┆ ---          ┆ ---               ┆ ---            │
│ ---                ┆ i64       ┆ u32         ┆ str          ┆ str               ┆ str            │
│ i64                ┆           ┆             ┆              ┆                   ┆                │
╞════════════════════╪═══════════╪═════════════╪══════════════╪═══════════════════╪════════════════╡
│ 894008343785500    ┆ 42102006 

## 步骤 5: 保存分析结果

In [6]:
# 保存完整作者画像
author_profile_path = Path("../parquet/author_profiling.parquet")
io.materialize_parquet(author_stats.lazy(), author_profile_path)
print(f"✅ 作者画像已保存: {author_profile_path}")

# 保存 Top 50
top_50_path = Path("../parquet/top_50_influencers.parquet")
io.materialize_parquet(top_50.lazy(), top_50_path)
print(f"✅ Top 50 已保存: {top_50_path}")

print(f"\n📊 数据概览:")
print(f"  总作者数: {author_stats.height:,}")
print(f"  影响力分层: 4 个层级")
print(f"  立场一致性分析: 完成")
print(f"  Top 50 高影响力: 已提取")

✅ 作者画像已保存: ../parquet/author_profiling.parquet
✅ Top 50 已保存: ../parquet/top_50_influencers.parquet

📊 数据概览:
  总作者数: 135
  影响力分层: 4 个层级
  立场一致性分析: 完成
  Top 50 高影响力: 已提取


## ✅ 作者画像分析完成！

**生成数据**:
- `author_profiling.parquet`: 完整的作者画像与立场一致性分析
- `top_50_influencers.parquet`: Top 50高影响力作者详情

**关键洞察**:
1. 影响力分层立场分布
2. 作者bio vs 推文立场一致性
3. 高影响力作者的叙事与情感偏好

**下一步**: 结合所有分析结果，构建综合Dashboard